In [1]:
# in reference to:
# https://towardsdatascience.com/scraping-reddit-with-praw-76efc1d1e1d9
# https://praw.readthedocs.io/en/latest/code_overview/praw_models.html
# https://www.reddit.com/wiki/search

In [1]:
from os.path import isfile
import praw
import pandas as pd
from time import sleep

# Get credentials from DEFAULT instance in praw.ini
# Use your own tokens
reddit = praw.Reddit(
    client_id="_x_YEbApcORK1g",
    client_secret="fjZtRV5hPbiLchwXZgSWAPD6nzo",
    user_agent="Scraper",
    username="Scraper3",
    password="Scraper3"
)


class SubredditScraper:

    def __init__(self, query, sort='new', lim=900, mode='w'):
        self.query = query
        self.sort = sort
        self.lim = lim
        self.mode = mode

        print(
            f'SubredditScraper instance created with values '
            f'query = {query}, sort = {sort}, lim = {lim}, mode = {mode}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit('all').search(self.query, sort='new', time_filter='month')
        elif self.sort == 'top':
            return self.sort, reddit.subreddit('all').search(self.query, sort='top', time_filter='month')
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit('all').search(self.query, sort='hot', time_filter='month')
        else:
            self.sort = 'relevance'
            print('Sort method was not recognized, defaulting to relevance.')
            return self.sort, reddit.subreddit('all').search(self.query, time_filter='month')

    def get_posts(self, printer=False):

        sub_dict = {
            'game_title': [], 'selftext': [], 'title': [], 'id': [], 'sorted_by': [],
            'num_comments': [], 'score': [], 'ups': [], 'downs': []}
        csv = f'game_titles_4.csv'

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        # Set csv_loaded to True if csv exists since you can't evaluate the
        # truth value of a DataFrame.
        df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

        print(f'csv = {csv}')
        print(f'After set_sort(), sort = {sort} and query = {self.query}')
        print(f'csv_loaded = {csv_loaded}')

        print(f'Collecting information from r/all for search term "{self.query}".')
        for post in subreddit:
            # Save any unique, non-stickied posts with descriptions to sub_dict.
            sub_dict['game_title'].append(self.query)
            sub_dict['selftext'].append(post.selftext)
            sub_dict['title'].append(post.title)
            sub_dict['id'].append(post.id)
            sub_dict['sorted_by'].append(sort)
            sub_dict['num_comments'].append(post.num_comments)
            sub_dict['score'].append(post.score)
            sub_dict['ups'].append(post.ups)
            sub_dict['downs'].append(post.downs)
            sleep(0.1)

        sub_dict_final = {
            'game_title': [self.query],  
            'score': [sum(sub_dict['score'])], 
            'ups': [sum(sub_dict['ups'])], 
            'downs': [sum(sub_dict['downs'])]}
        
        # print(sub_dict)
        new_df = pd.DataFrame(sub_dict_final)
        
        # decide whether or no to print
        if printer is True:
            print(
                f'{len(sub_dict)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')
            return

        # Add new_df to df if df exists then save it to a csv.
        if 'DataFrame' in str(type(df)) and self.mode == 'w':
            pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
            print(
                f'{len(new_df)} new posts collected and added to {csv}')
        elif self.mode == 'w':
            new_df.to_csv(csv, index=False)
            print(f'{len(new_df)} posts collected and saved to {csv}')
        else:
            print(
                f'{len(new_df)} posts were collected but they were not '
                f'added to {csv} because mode was set to "{self.mode}"')


#if __name__ == '__main__':
    #SubredditScraper('statistics', lim=997, mode='w', sort='new').get_posts()
    #SubredditScraper('Over Watch', lim=997, sort='new').get_posts()

In [2]:
all_titles = pd.read_csv("All_Games.csv", error_bad_lines=False)

b'Skipping line 252: expected 8 fields, saw 9\nSkipping line 284: expected 8 fields, saw 10\nSkipping line 396: expected 8 fields, saw 9\nSkipping line 609: expected 8 fields, saw 9\nSkipping line 610: expected 8 fields, saw 9\nSkipping line 628: expected 8 fields, saw 9\nSkipping line 696: expected 8 fields, saw 9\nSkipping line 1561: expected 8 fields, saw 9\nSkipping line 2536: expected 8 fields, saw 9\nSkipping line 2650: expected 8 fields, saw 9\nSkipping line 2726: expected 8 fields, saw 9\nSkipping line 2773: expected 8 fields, saw 9\nSkipping line 2891: expected 8 fields, saw 11\nSkipping line 2892: expected 8 fields, saw 10\nSkipping line 2983: expected 8 fields, saw 9\nSkipping line 3005: expected 8 fields, saw 9\nSkipping line 3094: expected 8 fields, saw 10\nSkipping line 3119: expected 8 fields, saw 9\nSkipping line 3141: expected 8 fields, saw 10\nSkipping line 3151: expected 8 fields, saw 13\nSkipping line 3171: expected 8 fields, saw 17\nSkipping line 3289: expected 8 f

In [3]:
# [[0, 1283], [1283, 2566], [2566, 3849], [3849, 5132], [5132, 6417]]

for i in range(5132, 6417):
    print('Scraping: (' + str(i) + ')')
    title = all_titles.title[i]
    SubredditScraper(str(title), lim=997, sort='new').get_posts()

Scraping: (5132)
SubredditScraper instance created with values query = Fighters Uncaged, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Fighters Uncaged
csv_loaded = 0
1 posts collected and saved to game_titles_4.csv
Scraping: (5133)
SubredditScraper instance created with values query = Final Fantasy XI: Seekers of Adoulin, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Final Fantasy XI: Seekers of Adoulin
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5134)
SubredditScraper instance created with values query = Final Fantasy XI: Vana'diel Collection 2008, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Final Fantasy XI: Vana'diel Collection 2008
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5135)
SubredditScraper instance created with values query = Final Fantasy XI: Wings of

1 new posts collected and added to game_titles_4.csv
Scraping: (5156)
SubredditScraper instance created with values query = FunTown Mahjong, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = FunTown Mahjong
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5157)
SubredditScraper instance created with values query = Fusion: Genesis, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Fusion: Genesis
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5158)
SubredditScraper instance created with values query = Fuzion Frenzy 2, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Fuzion Frenzy 2
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5159)
SubredditScraper instance created with values query = Galaga, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After

1 new posts collected and added to game_titles_4.csv
Scraping: (5180)
SubredditScraper instance created with values query = Guardian Heroes, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Guardian Heroes
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5181)
SubredditScraper instance created with values query = Guilty Gear 2: Overture, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Guilty Gear 2: Overture
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5182)
SubredditScraper instance created with values query = Guitar Hero II, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Guitar Hero II
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5183)
SubredditScraper instance created with values query = Gun, sort = new, lim = 997, mode = w
csv = game_titles_

1 new posts collected and added to game_titles_4.csv
Scraping: (5205)
SubredditScraper instance created with values query = Hexodius, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Hexodius
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5206)
SubredditScraper instance created with values query = High School Musical 3: Senior Year DANCE!, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = High School Musical 3: Senior Year DANCE!
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5207)
SubredditScraper instance created with values query = History – Civil War: Secret Missions, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = History – Civil War: Secret Missions
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5208)
SubredditScraper instance created with valu

1 new posts collected and added to game_titles_4.csv
Scraping: (5229)
SubredditScraper instance created with values query = Iron Brigade, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Iron Brigade
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5230)
SubredditScraper instance created with values query = Iron Sky: Invasion, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Iron Sky: Invasion
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5231)
SubredditScraper instance created with values query = Islands of Wakfu, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Islands of Wakfu
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (5232)
SubredditScraper instance created with values query = James Cameron's Avatar: The Game, sort = new, lim = 997, mode = w
c

...

1 new posts collected and added to game_titles_4.csv
Scraping: (6318)
SubredditScraper instance created with values query = Subaeria, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Subaeria
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6319)
SubredditScraper instance created with values query = Subject 13, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Subject 13
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6320)
SubredditScraper instance created with values query = Sunset Overdrive, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Sunset Overdrive
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6321)
SubredditScraper instance created with values query = Super Blackjack Battle II Turbo Edition, sort = new, lim = 997, mode = w
csv = game_titles_

1 new posts collected and added to game_titles_4.csv
Scraping: (6343)
SubredditScraper instance created with values query = Tempest 4000, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Tempest 4000
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6344)
SubredditScraper instance created with values query = Tesla Effect: A Tex Murphy Adventure, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Tesla Effect: A Tex Murphy Adventure
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6345)
SubredditScraper instance created with values query = Tetra's Escape, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Tetra's Escape
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6346)
SubredditScraper instance created with values query = theHunter: Call of the Wild, sort =

1 new posts collected and added to game_titles_4.csv
Scraping: (6367)
SubredditScraper instance created with values query = Tunic, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Tunic
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6368)
SubredditScraper instance created with values query = Twin Mirror, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Twin Mirror
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6369)
SubredditScraper instance created with values query = Tyler: Model 005, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Tyler: Model 005
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6370)
SubredditScraper instance created with values query = Unexplored, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = ne

1 new posts collected and added to game_titles_4.csv
Scraping: (6391)
SubredditScraper instance created with values query = Western Press, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Western Press
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6392)
SubredditScraper instance created with values query = Winx Club: Alfea Butterflix Adventures, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Winx Club: Alfea Butterflix Adventures
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6393)
SubredditScraper instance created with values query = We Are Doomed, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = We Are Doomed
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6394)
SubredditScraper instance created with values query = We Are the Dwarves, sort = new,

1 new posts collected and added to game_titles_4.csv
Scraping: (6415)
SubredditScraper instance created with values query = Zombie Pinball, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Zombie Pinball
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
Scraping: (6416)
SubredditScraper instance created with values query = Zombie Vikings, sort = new, lim = 997, mode = w
csv = game_titles_4.csv
After set_sort(), sort = new and query = Zombie Vikings
csv_loaded = 1
1 new posts collected and added to game_titles_4.csv
